### Statistical analysis ###

In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt

### APD ###

In [22]:
importlib.reload(dr_a)
importlib.reload(dt)


# BPM
apd_metric_folder = dr_a.Paths.METRICS
phases = dr_a.Phases.PHASES_LIST
for phase in phases:
    files = glob.glob(os.path.join(apd_metric_folder, f"bpm_{phase}*.csv"))

    dfs = [pd.read_csv(file, index_col=[0]).to_numpy() for file in files]
    for i in range(len(dfs)):
        df = dfs[i]
        subjects = df[:, 0]
        mean_col = np.nanmean(df[:, 1:], axis=1)
        dfs[i] = pd.concat()


[array([139.1344351 ,  89.44622053,  88.66834   ,  91.66919221,
        56.75626218, 128.4851489 , 128.32314021,  90.95111876,
        76.47293597, 155.10627792, 151.37376221, 130.46311811,
       102.74170331,  90.48368311,  72.00263382, 151.66223151,
       123.0660484 , 178.75293892, 481.6468254 ,  81.0457001 ,
       101.97705398, 104.4862232 ,  80.14941114, 125.74355588,
       105.55563526,  99.42475805]), array([ 84.24037119, 155.83306874,  75.01800554,  86.92257352,
        92.22704884,  78.35982366,  83.74760191,  90.74676667,
       286.45156819,  89.30731635, 122.76793393,  69.21890336,
       103.04175561,  91.00085865, 129.66088539,  80.99582944,
       101.54542782, 145.60786086,  72.93077918,  77.09211767,
        77.24300475,  65.02294975,  97.53007361, 112.80658391,
        74.58976601, 100.65416193])]
[array([156.57137165,  93.38074489,  86.88896085,  99.26500173,
        61.28042635, 147.35236177, 126.66105067,  98.43258167,
        87.34633676, 152.29736591,  72.111